# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

-----
<img src = 'assets/Flowers.png' width=500 px>

O projeto é dividido em várias etapas:

* Carregar e pré-processar o conjunto de dados da imagem
* Treine o classificador de imagens em seu conjunto de dados
* Use o classificador treinado para prever o conteúdo da imagem

Nós vamos guiá-lo através de cada parte que você implementará em Python.

Quando você concluir este projeto, você terá um aplicativo que pode ser treinado em qualquer conjunto de imagens rotuladas. Aqui, sua rede aprenderá sobre flores e terminará como um aplicativo de linha de comando. Mas o que você faz com suas novas habilidades depende da sua imaginação e esforço na construção de um conjunto de dados. Por exemplo, imagine um aplicativo em que você tira uma foto de um carro, informa a marca e o modelo e procura informações sobre ele. Vá criar seu próprio conjunto de dados e faça algo novo.

A primeira é importar os pacotes que você precisa. É uma boa prática manter todas as importações no início do seu código. À medida que você trabalha nesse bloco de anotações e descobre que precisa importar um pacote, adicione a importação aqui.

In [1]:
# Imports here
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
import numpy as np


## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). You can [download the data here](https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip). The dataset is split into two parts, training and validation. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. If you use a pre-trained network, you'll also need to make sure the input data is resized to 224x224 pixels as required by the networks.

The validation set is used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks available from `torchvision` were trained on the ImageNet dataset where each color channel was normalized separately. For both sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.

---
Aqui você usará o `torchvision` para carregar os dados ([documentation] (http://pytorch.org/docs/0.3.0/torchvision/index.html)). Você pode [baixar os dados aqui] (https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip). O conjunto de dados é dividido em duas partes, treinamento e validação. Para o treinamento, você desejará aplicar transformações como dimensionamento, recorte e inversão aleatórios. Isso ajudará a generalizar a rede levando a um melhor desempenho. Se você usar uma rede pré-treinada, também precisará garantir que os dados de entrada sejam redimensionados para 224x224 pixels, conforme exigido pelas redes.

O conjunto de validação é usado para medir o desempenho do modelo em dados que ainda não foram vistos. Para isso, você não deseja nenhuma transformação de escala ou rotação, mas precisará redimensioná-las e recortar as imagens no tamanho apropriado.

As redes pré-treinadas disponíveis no `torchvision` foram treinadas no banco de dados ImageNet, onde cada canal de cores foi normalizado separadamente. Para ambos os conjuntos, você precisará normalizar as médias e os desvios padrão das imagens para o que a rede espera. Para os meios, é `[0.485, 0.456, 0.406]` e para os desvios padrão `[0.229, 0.224, 0.225]`, calculados a partir das imagens do ImageNet. Esses valores irão deslocar cada canal de cor para ser centralizado em 0 e variar de -1 a 1.

In [2]:
data_dir = 'flower_data'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'

In [3]:
from torchvision import datasets, transforms, models

# TODO: Define your transforms for the training and validation sets
data_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])


# TODO: Load the datasets with ImageFolder
image_datasets = datasets.ImageFolder(train_dir, transform=data_transforms)
valid_datasets = datasets.ImageFolder(valid_dir, transform=data_transforms)

# TODO: Using the image datasets and the trainforms, define the dataloaders
trainloader = torch.utils.data.DataLoader(image_datasets, batch_size=32, shuffle=True)
validloader = torch.utils.data.DataLoader(valid_datasets, batch_size=32, shuffle=True)
#trainloader = torch.utils.data.DataLoader(image_datasets, shuffle=True)
#validloader = torch.utils.data.DataLoader(valid_datasets, shuffle=True)

print(len(trainloader))
print(len(validloader))


205
26


### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

---

Você também precisará carregar um mapeamento do rótulo da categoria para o nome da categoria. Você pode encontrar isto no arquivo `cat_to_name.json`. É um objeto JSON que você pode ler com o módulo [`json`] (https://docs.python.org/2/library/json.html). Isso lhe dará um dicionário mapeando as categorias codificadas por números inteiros para os nomes reais das flores.

In [4]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)
    print(cat_to_name)

{'21': 'fire lily', '3': 'canterbury bells', '45': 'bolero deep blue', '1': 'pink primrose', '34': 'mexican aster', '27': 'prince of wales feathers', '7': 'moon orchid', '16': 'globe-flower', '25': 'grape hyacinth', '26': 'corn poppy', '79': 'toad lily', '39': 'siam tulip', '24': 'red ginger', '67': 'spring crocus', '35': 'alpine sea holly', '32': 'garden phlox', '10': 'globe thistle', '6': 'tiger lily', '93': 'ball moss', '33': 'love in the mist', '9': 'monkshood', '102': 'blackberry lily', '14': 'spear thistle', '19': 'balloon flower', '100': 'blanket flower', '13': 'king protea', '49': 'oxeye daisy', '15': 'yellow iris', '61': 'cautleya spicata', '31': 'carnation', '64': 'silverbush', '68': 'bearded iris', '63': 'black-eyed susan', '69': 'windflower', '62': 'japanese anemone', '20': 'giant white arum lily', '38': 'great masterwort', '4': 'sweet pea', '86': 'tree mallow', '101': 'trumpet creeper', '42': 'daffodil', '22': 'pincushion flower', '2': 'hard-leaved pocket orchid', '54': 's

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. If you want to talk through it with someone, chat with your fellow students! You can also ask questions on the forums or join the instructors in office hours.

Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

----
# Construindo e treinando o classificador

Agora que os dados estão prontos, é hora de construir e treinar o classificador. Como de costume, você deve usar um dos modelos pré-concebidos de `torchvision.models` para obter os recursos da imagem. Crie e treine um novo classificador de feed-forward usando esses recursos.

Nós vamos deixar essa parte para você. Se você quiser conversar com alguém, converse com seus colegas! Você também pode fazer perguntas nos fóruns ou se juntar aos instrutores em horário de expediente.

Consulte a [rubrica] (https://review.udacity.com/#!/rubrics/1663/view) para obter orientação sobre como concluir com êxito esta seção. Coisas que você precisa fazer:

* Carregue uma [rede pré-treinada] (http://pytorch.org/docs/master/torchvision/models.html) (Se você precisar de um ponto de partida, as redes VGG funcionam muito bem e são fáceis de usar)
* Definir uma nova rede de feed-forward não treinada como um classificador, usando ativações de ReLU e dropout
* Treinar as camadas do classificador usando backpropagation usando a rede pré-treinada para obter os recursos
* Acompanhe a perda e precisão no conjunto de validação para determinar os melhores hiperparâmetros

Nós deixamos uma célula aberta para você abaixo, mas use quantos você precisar. Nosso conselho é dividir o problema em partes menores que você pode executar separadamente. Verifique se cada parte está fazendo o que você espera e, em seguida, passe para a próxima. Você provavelmente descobrirá que, ao trabalhar em cada parte, precisará voltar e modificar seu código anterior. Isso é totalmente normal!

Ao treinar, certifique-se de atualizar apenas os pesos da rede de feed-forward. Você deve ser capaz de obter a precisão de validação acima de 70% se você construir tudo certo. Certifique-se de tentar diferentes hyperparameters (taxa de aprendizagem, unidades no classificador, épocas, etc) para encontrar o melhor modelo. Salve esses hiperparâmetros para usar como valores padrão na próxima parte do projeto.

In [5]:
# TODO: Build and train your network
"""
from collections import OrderedDict
model = models.densenet121(pretrained=True)

for param in model.parameters():
    param.requires_grad = False


classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(1024, 500)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(500, 2)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
    
model.classifier = classifier


criterion = nn.NLLLoss()
# Only train the classifier parameters, feature parameters are frozen
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for ii, (inputs, labels) in enumerate(train_dataloaders):
    print("Teste")
'''
    # Move input and label tensors to the GPU
    inputs, labels = inputs.to(device), labels.to(device)

    #start = time.time()

    outputs = model.forward(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    if ii==3:
        break
'''
"""

'\nfrom collections import OrderedDict\nmodel = models.densenet121(pretrained=True)\n\nfor param in model.parameters():\n    param.requires_grad = False\n\n\nclassifier = nn.Sequential(OrderedDict([\n                          (\'fc1\', nn.Linear(1024, 500)),\n                          (\'relu\', nn.ReLU()),\n                          (\'fc2\', nn.Linear(500, 2)),\n                          (\'output\', nn.LogSoftmax(dim=1))\n                          ]))\n    \nmodel.classifier = classifier\n\n\ncriterion = nn.NLLLoss()\n# Only train the classifier parameters, feature parameters are frozen\noptimizer = optim.Adam(model.classifier.parameters(), lr=0.001)\n\ndevice = torch.device("cuda" if torch.cuda.is_available() else "cpu")\nmodel.to(device)\n\nfor ii, (inputs, labels) in enumerate(train_dataloaders):\n    print("Teste")\n\'\'\'\n    # Move input and label tensors to the GPU\n    inputs, labels = inputs.to(device), labels.to(device)\n\n    #start = time.time()\n\n    outputs = model.f

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model = models.densenet121(pretrained=True)
#model = models.vgg11(pretrained=True)
model = models.vgg19(pretrained=True)

# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False
    
#model.classifier = nn.Sequential(nn.Linear(1024, 256),
model.classifier = nn.Sequential(nn.Linear(25088, 256),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(256, 102),
                                 nn.LogSoftmax(dim=1))

criterion = nn.NLLLoss()

# Only train the classifier parameters, feature parameters are frozen
optimizer = optim.Adam(model.classifier.parameters(), lr=0.003)

model.to(device)


Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to C:\Users\Rafael/.torch\models\vgg19-dcbb9e9d.pth
100.0%


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (17): ReLU(inplace)

In [19]:
epochs = 1
steps = 0
running_loss = 0
print_every = 1
for epoch in range(epochs):
    for inputs, labels in trainloader:
        #print(inputs)
        
        steps += 1
        # Move input and label tensors to the default device
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        logps = model.forward(inputs)
        #print(logps)
        
        loss = criterion(logps, labels)
        
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for inputs, labels in validloader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    logps = model.forward(inputs)
                    batch_loss = criterion(logps, labels)
                    
                    test_loss += batch_loss.item()
                    
                    # Calculate accuracy
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                    
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(validloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(validloader):.3f}")
            running_loss = 0
            model.train()

checkpoint = {'input_size': 784,
              'output_size': 102,
              'hidden_layers': [each.out_features for each in model.hidden_layers],
              'state_dict': model.state_dict()}

#torch.save(checkpoint, 'checkpoint.pth')            
torch.save(model.state_dict(), 'checkpoint.pth')

Epoch 1/5.. Train loss: 4.738.. Test loss: 8.018.. Test accuracy: 0.039
Epoch 1/5.. Train loss: 9.337.. Test loss: 8.962.. Test accuracy: 0.016
Epoch 1/5.. Train loss: 10.692.. Test loss: 8.718.. Test accuracy: 0.050
Epoch 1/5.. Train loss: 10.735.. Test loss: 7.082.. Test accuracy: 0.097
Epoch 1/5.. Train loss: 6.168.. Test loss: 6.730.. Test accuracy: 0.109
Epoch 1/5.. Train loss: 6.403.. Test loss: 6.622.. Test accuracy: 0.087
Epoch 1/5.. Train loss: 6.623.. Test loss: 6.205.. Test accuracy: 0.089
Epoch 1/5.. Train loss: 8.277.. Test loss: 5.436.. Test accuracy: 0.097
Epoch 1/5.. Train loss: 6.106.. Test loss: 4.948.. Test accuracy: 0.086


KeyboardInterrupt: 

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

---

## Salve o ponto de verificação

Agora que sua rede está treinada, salve o modelo para que você possa carregá-lo posteriormente para fazer previsões. Você provavelmente quer salvar outras coisas, como o mapeamento de classes para índices que você obtém de um dos conjuntos de dados da imagem: `image_datasets ['train']. Class_to_idx`. Você pode anexar isso ao modelo como um atributo que facilita a inferência mais tarde.

`` `model.class_to_idx = image_datasets ['train']. class_to_idx```

Lembre-se de que você vai querer reconstruir completamente o modelo mais tarde para poder usá-lo para inferência. Certifique-se de incluir qualquer informação que você precisa no checkpoint. Se você quiser carregar o modelo e continuar treinando, será necessário salvar o número de épocas, bem como o estado do otimizador, `optimizer.state_dict`. Você provavelmente vai querer usar esse modelo treinado na próxima parte do projeto, então é melhor salvá-lo agora.

In [ ]:
# TODO: Save the checkpoint 

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [ ]:
# TODO: Write a function that loads a checkpoint and rebuilds the model

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    
    # TODO: Process a PIL image for use in a PyTorch model

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    # TODO: Implement the code to predict the class from an image file

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the validation accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# TODO: Display an image along with the top 5 classes